In [1]:
import numpy as np
import tensorflow as tf
import logging

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ayeghiazaryan/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from transformer_modules import *

In [3]:
np.random.seed(20)

In [4]:
# q0 = np.random.randn(5,10, 1,20).astype(np.float32)
# k0 = np.random.randn(5,10, 1,20).astype(np.float32)
# v0 = np.random.randn(5,10, 1,20).astype(np.float32)

# q = tf.placeholder_with_default(input = q0, shape=[None, 10, 1, 20])
# k = tf.placeholder_with_default(input = k0, shape=[None, 10, 1, 20])
# v = tf.placeholder_with_default(input = v0, shape=[None, 10, 1, 20])

In [5]:
# attn = ScaledDotProductAttention()

In [6]:
q0 = np.random.randn(5,10, 100).astype(np.float32)
k0 = np.random.randn(5,10, 100).astype(np.float32)
v0 = np.random.randn(5,10, 100).astype(np.float32)

q = tf.placeholder_with_default(input = q0, shape=[None, 10, 100])
k = tf.placeholder_with_default(input = k0, shape=[None, 10, 100])
v = tf.placeholder_with_default(input = v0, shape=[None, 10, 100])

In [7]:
# attn_head = MultiHeadAttnetion(100, 5)
# enc = EncoderBlock(100, 20, n_heads=5, d_ff=500)
enc = TransformerEncoder(n_blocks=3, d_model=100, n_heads=5, d_ff=500)

In [8]:
# x = attn_head(q, k, v)
enc_out = enc(q)

Encoder, input size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
Encoder, attention output size=(?, 10, 100)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Encoder, output size=(?, 10, 100)
Encoder, input size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
Encoder, attention output size=(?, 10, 100)
Encoder, output size=(?, 10, 100)
Encoder, input size=(?, 10, 100)
ScaledDotProductAttention, attention weights size

In [9]:
dec = TransformerDecoder(n_blocks=3, d_model=100, n_heads=5, d_ff=500)

In [10]:
dec_out = dec(q, enc_out)

ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 20)
Multihead Attention, output size=(?, 10, 100)
ScaledDotProductAttention, attention weights size=(?, 5, 10, 10)
ScaledDotProductAttention, attention output size=(?, 10, 5, 

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
x_outp = sess.run(dec_out)

In [13]:
x_outp.shape

(5, 10, 100)

In [23]:
aa = tf.reshape(tf.Variable(initial_value=np.arange(1000)), (5,10,20))

In [24]:
aa

<tf.Tensor 'Reshape_5:0' shape=(5, 10, 20) dtype=int64>

In [25]:
bb = tf.reshape(aa, (5,-1,2,10))

In [26]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [27]:
aa_s=sess.run(aa)

In [28]:
aa_r = sess.run(bb)

In [29]:
aa_r.shape

(5, 10, 2, 10)

In [42]:
aa_r[4,7,:,:]

array([[940, 941, 942, 943, 944, 945, 946, 947, 948, 949],
       [950, 951, 952, 953, 954, 955, 956, 957, 958, 959]])

In [43]:
aa_s[4,7,:]

array([940, 941, 942, 943, 944, 945, 946, 947, 948, 949, 950, 951, 952,
       953, 954, 955, 956, 957, 958, 959])